In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [4]:
file_path = "photos/listochek.png"
img = cv2.imread(file_path)

data = np.array(img)
data_list = img.tolist()

# 1) Зсув

In [6]:
def moving(data):
    a = 10
    b = 20
    result = np.zeros(data.shape, dtype=np.uint8)

    for y in range(data.shape[0]):
        for x in range(data.shape[1]):
            X_plus = x + a
            Y_plus = y + b
            if 0 <= X_plus < data.shape[1] and 0 <= Y_plus < data.shape[0]:
                result[Y_plus, X_plus] = data[y, x]
    return result

In [7]:
img_shift = moving(data)
cv2.imwrite('result/listochek1.png', img_shift)

True

# 2) Інверсія

In [8]:
def inversion(image):

    kernel = np.array([[0, 0, 0], [0, -1, 0], [0, 0, 0]])  
    kernel = np.flipud(np.fliplr(kernel)) 


    output = np.zeros_like(image, dtype=np.float64)
    padded_image = np.pad(image, ((kernel.shape[0] // 2,), (kernel.shape[1] // 2,)), mode='constant', constant_values=0)

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            region = padded_image[i:i + kernel.shape[0], j:j + kernel.shape[1]]
            output[i, j] = np.sum(region * kernel)

    return output.astype(np.uint8)

In [9]:
image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE) 

img_inversion_3 = inversion(image)
cv2.imwrite('result/listochek2.png', img_inversion_3)

True

# 3) Згладжування по Гауссу

In [15]:
def gaussian_blur(image, kernel_s, sigma=1):
    def dnorm(x, mean=0, std_dev=1):
        return (1 / (std_dev * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mean) / std_dev) ** 2)

    kernel_1D = np.linspace(-(kernel_s // 2), kernel_s // 2, kernel_s)
    for i in range(kernel_s):
        kernel_1D[i] = dnorm(kernel_1D[i], 0, sigma)

    kernel_2D = np.outer(kernel_1D.T, kernel_1D.T)
    kernel_2D /= kernel_2D.sum()

    pad_size = kernel_s // 2
    result = np.zeros_like(image)

    if len(image.shape) == 2: 
        image_padded = np.pad(image, ((pad_size, pad_size), (pad_size, pad_size)), mode='constant')
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                region = image_padded[i:i + kernel_s, j:j + kernel_s]
                result[i, j] = np.sum(region * kernel_2D)
    else: 
        for channel in range(image.shape[2]):
            img_w_padd = np.pad(image[..., channel], ((pad_size, pad_size), (pad_size, pad_size)), mode='constant')
            for i in range(image.shape[0]):
                for j in range(image.shape[1]):
                    region = img_w_padd[i:i + kernel_s, j:j + kernel_s]
                    result[i, j, channel] = np.sum(region * kernel_2D)

    return result


In [16]:
gaussian_img = gaussian_blur(img, 11, sigma=1)
cv2.imwrite('result/listochek3.png', gaussian_img)

True

# 4) Розмиття "рух по дiагоналi"

In [19]:
def diagonal_kernel(size):
    kernel = np.eye(size, dtype=np.float32)
    return kernel / np.sum(kernel)

def diagonal_motion(image, kernel_size):
    kernel = diagonal_kernel(kernel_size)
    pad = kernel_size // 2
    result = np.zeros_like(image)

    for ch in range(image.shape[2]):
        padded = np.pad(image[..., ch], pad_width=((pad, pad), (pad, pad)), mode='constant')
        for x in range(image.shape[0]):
            for y in range(image.shape[1]):
                region = padded[x:x + kernel_size, y:y + kernel_size]
                result[x, y, ch] = np.sum(region * kernel)

    return result


In [20]:
img_diagonal_motion = diagonal_motion(img, 8)
cv2.imwrite('result/listochek4.png', img_diagonal_motion)

True

# 5) Пiдвищення рiзкостi

In [23]:
sharpen = np.array([[0, -1, 0],
                    [-1, 5, -1],
                    [0, -1, 0]])

def sharpening(image, kernel):
    pad = kernel.shape[0] // 2  # Размер паддинга
    padded_image = np.pad(image, ((pad, pad), (pad, pad), (0, 0)), mode='constant')
    output_image = np.zeros_like(image, dtype=np.uint8)  # Указываем тип сразу

    height, width, channels = image.shape

    for row in range(height):
        for col in range(width):
            for channel in range(channels):
                # Извлечение региона и применение свёртки
                region = padded_image[row:row + kernel.shape[0], col:col + kernel.shape[1], channel]
                pixel_value = np.sum(region * kernel)
                output_image[row, col, channel] = np.clip(pixel_value, 0, 255)  # Ограничиваем значения

    return output_image


In [24]:
sharpen_img = sharpening(img, sharpen)
cv2.imwrite('result/listochek5.png', sharpen_img)

True

# 6) Фiльтр Собеля

In [25]:
def sobel_filter(image):
    horizontal = np.array([[-1, 0, 1],
                           [-2, 0, 2],
                           [-1, 0, 1]])
    
    vertical = np.array([[-1, -2, -1],
                         [ 0,  0,  0],
                         [ 1,  2,  1]])

    height, width, channels = image.shape
    pad_size = 1
    padded_image = np.pad(image, ((pad_size, pad_size), (pad_size, pad_size), (0, 0)), mode='constant')
    
    result = np.zeros_like(image, dtype=np.uint8)

    for c in range(channels):  
        for i in range(height):
            for j in range(width):
                region = padded_image[i:i + 3, j:j + 3, c]
                grad_x = np.sum(region * horizontal)
                grad_y = np.sum(region * vertical)
                magnitude = np.sqrt(grad_x**2 + grad_y**2)
                result[i, j, c] = np.clip(magnitude, 0, 255)

    return result

In [26]:
sobel_result = sobel_filter(img)

# Save the result
cv2.imwrite('result/listochek6.png', sobel_result)

True

# 7) Фiльтр границi

In [27]:
def border_detection(image):
    kernel = np.array([[-1, -1, -1],
                       [-1,  8, -1],
                       [-1, -1, -1]])
    
    height, width, channels = image.shape
    pad_size = kernel.shape[0] // 2
    padded_image = np.pad(image, ((pad_size, pad_size), (pad_size, pad_size), (0, 0)), mode='constant')
    
    output_image = np.zeros_like(image, dtype=np.uint8)

    for channel in range(channels):  
        for row in range(height):
            for col in range(width):
                region = padded_image[row:row + kernel.shape[0], col:col + kernel.shape[1], channel]
                pixel_value = np.sum(region * kernel)
                output_image[row, col, channel] = np.clip(pixel_value, 0, 255)

    return output_image


In [39]:
img_bord = border_detection(img)
cv2.imwrite('result/listochek7.png', img_bord)

True

# 8) Який-небудь фiльтр який я вигадав сам

In [40]:
def emboss_filter(image):
    kernel = np.array([[-1, -1, -1],
                       [-1,  8, -1],
                       [-1, -1, -1]])
    
    height, width, channels = image.shape
    pad_size = kernel.shape[0] // 2

    padded_image = np.pad(image, ((pad_size, pad_size), (pad_size, pad_size), (0, 0)), mode='constant')
    
    output_image = np.zeros_like(image, dtype=np.float64)

    for row in range(height):
        for col in range(width):
            for channel in range(channels):
                region = padded_image[row:row + kernel.shape[0], col:col + kernel.shape[1], channel]
                pixel_value = np.sum(region * kernel)
                output_image[row, col, channel] = np.clip(pixel_value, 0, 255)

    return output_image.astype(np.uint8)


In [41]:
emboss_img = emboss_filter(img)
cv2.imwrite('result/listochek8.png', emboss_img)

True